# 29.1 THE PROBLEM

# 29.2 A RECIPE APPLICATION

In [8]:
abstract class Food(val name: String) {
  override def toString = name
}

defined class Food

In [9]:
class Recipe(
  val name: String,
  val ingredients: List[Food],
  val instructions: String
) {
  override def toString = name
}

defined class Recipe

In [10]:
object Apple extends Food("Apple")
object Orange extends Food("Orange")
object Cream extends Food("Cream")
object Sugar extends Food("Sugar")

object FruitSalad extends Recipe(
  "fruit salad",
  List(Apple, Orange, Cream, Sugar),
  "Stir it all together."
)

defined object Apple
defined object Orange
defined object Cream
defined object Sugar
defined object FruitSalad

In [ ]:
object SimpleDatabase {
  def allFoods = List(Apple, Orange, Cream, Sugar)

  def foodNamed(name: String): Option[Food] =
    allFoods.find(_.name == name)

  def allRecipes: List[Recipe] = List(FruitSalad)

  case class FoodCategory(name: String, foods: List[Food])

  private var categories = List(
    FoodCategory("fruits", List(Apple, Orange)),
    FoodCategory("misc", List(Cream, Sugar)))

  def allCategories = categories
}

object SimpleBrowser {
  def recipesUsing(food: Food) =
    SimpleDatabase.allRecipes.filter(recipe =>
      recipe.ingredients.contains(food))

  def displayCategory(category: SimpleDatabase.FoodCategory) = {
    println(category)
  }
}

In [11]:
val apple = SimpleDatabase.foodNamed("Apple").get
SimpleBrowser.recipesUsing(apple)

apple: $sess.cmd3.wrapper.cmd0.Food = Apple
res10_1: List[$sess.cmd1.wrapper.cmd0.Recipe] = List(fruit salad)

# 29.3 ABSTRACTION

In [15]:
abstract class Database {
  def allFoods: List[Food]

  def allRecipes: List[Recipe]

  def foodNamed(name: String) =
    allFoods.find(f => f.name == name)

  case class FoodCategory(name: String, foods: List[Food])

  def allCategories: List[FoodCategory]
}

defined class Database

In [16]:
abstract class Browser {
  val database: Database

  def recipesUsing(food: Food) =
    database.allRecipes.filter(recipe =>
      recipe.ingredients.contains(food))

  def displayCategory(category: database.FoodCategory) = {
    println(category)
  }
}

defined class Browser

In [17]:
object SimpleDatabase extends Database {
  def allFoods = List(Apple, Orange, Cream, Sugar)

  def allRecipes: List[Recipe] = List(FruitSalad)

  private var categories = List(
    FoodCategory("fruits", List(Apple, Orange)),
    FoodCategory("misc", List(Cream, Sugar)))

  def allCategories = categories
}

defined object SimpleDatabase

In [18]:
object SimpleBrowser extends Browser {
  val database = SimpleDatabase
}

defined object SimpleBrowser

In [19]:
val apple = SimpleDatabase.foodNamed("Apple").get
SimpleBrowser.recipesUsing(apple)

apple: Food = Apple
res18_1: List[Recipe] = List(fruit salad)

In [20]:
object StudentDatabase extends Database {

  object FrozenFood extends Food("FrozenFood")

  object HeatItUp extends Recipe(
    "heat it up",
    List(FrozenFood),
    "Microwave the 'food' for 10 minutes.")

  def allFoods = List(FrozenFood)

  def allRecipes = List(HeatItUp)

  def allCategories = List(
    FoodCategory("edible", List(FrozenFood)))
}

object StudentBrowser extends Browser {
  val database = StudentDatabase
}

defined object StudentDatabase
defined object StudentBrowser

# 29.4 SPLITTING MODULES INTO TRAITS

In [21]:
trait FoodCategories {
  case class FoodCategory(name: String, foods: List[Food])

  def allCategories: List[FoodCategory]
}

defined trait FoodCategories

In [22]:
abstract class Database extends FoodCategories {
  def allFoods: List[Food]

  def allRecipes: List[Recipe]

  def foodNamed(name: String) =
    allFoods.find(f => f.name == name)
}

defined class Database

In [23]:
trait SimpleFoods {
  object Pear extends Food("Pear")
  def allFoods = List(Apple, Pear)
  def allCategories = Nil
}

trait SimpleRecipes {
  this: SimpleFoods =>
  
  object FruitSalad extends Recipe(
    "fruit salad",
    List(Apple, Pear), // Now Pear is in scope
    "Mix it all together."
  )
  def allRecipes = List(FruitSalad)
}

object SimpleDatabase extends Database
  with SimpleFoods with SimpleRecipes

defined trait SimpleFoods
defined trait SimpleRecipes
defined object SimpleDatabase

# 29.5 RUNTIME LINKING

In [28]:
import $ivy.`com.lihaoyi::os-lib:0.9.1`
println(os.read(os.pwd / "ch29" / "GotApples1.scala"))

/// Food

abstract class Food(val name: String) {
  override def toString = name
}

/// Recipe

class Recipe(
  val name: String,
  val ingredients: List[Food],
  val instructions: String
) {
  override def toString = name
}


/// Database

trait FoodCategories {
  case class FoodCategory(name: String, foods: List[Food])

  def allCategories: List[FoodCategory]
}

abstract class Database extends FoodCategories {
  def allFoods: List[Food]

  def allRecipes: List[Recipe]

  def foodNamed(name: String) =
    allFoods.find(f => f.name == name)
}

/// Browser

abstract class Browser {
  val database: Database

  def recipesUsing(food: Food) =
    database.allRecipes.filter(recipe =>
      recipe.ingredients.contains(food))

  def displayCategory(category: database.FoodCategory) = {
    println(category)
  }
}

/// StudentDatabase, StudentBrowser

object StudentDatabase extends Database {

  object FrozenFood extends Food("FrozenFood")

  object HeatItUp extends Recipe(
    "heat it up",
  

import $ivy.$                          


In [29]:
os.proc("scala", "ch29/GotApples1.scala" , "simple").call().out.trim()

res28: String = "fruit salad"

In [30]:
os.proc("scala", "ch29/GotApples1.scala" , "student").call().out.trim()

res29: String = ""

# 29.6 TRACKING MODULE INSTANCES

In [31]:
val category = StudentDatabase.allCategories.head

category: StudentDatabase.FoodCategory = FoodCategory(
  name = "edible",
  foods = List(FrozenFood)
)

In [31]:
// SimpleBrowser.displayCategory(category)

cmd31.sc:1: type mismatch;
 found   : ammonite.$sess.cmd30.wrapper.cmd19.StudentDatabase.FoodCategory
 required: cmd31.this.cmd17.SimpleBrowser.database.FoodCategory
val res31 = SimpleBrowser.displayCategory(category)
                                          ^Compilation Failed

: 

In [32]:
println(os.read(os.pwd / "ch29" / "GotApples2.scala"))

/// Food

abstract class Food(val name: String) {
  override def toString = name
}

/// Recipe

class Recipe(
  val name: String,
  val ingredients: List[Food],
  val instructions: String
) {
  override def toString = name
}


/// Database

trait FoodCategories {
  case class FoodCategory(name: String, foods: List[Food])

  def allCategories: List[FoodCategory]
}

abstract class Database extends FoodCategories {
  def allFoods: List[Food]

  def allRecipes: List[Recipe]

  def foodNamed(name: String) =
    allFoods.find(f => f.name == name)
}

/// Browser

abstract class Browser {
  val database: Database

  def recipesUsing(food: Food) =
    database.allRecipes.filter(recipe =>
      recipe.ingredients.contains(food))

  def displayCategory(category: database.FoodCategory) = {
    println(category)
  }
}

/// StudentDatabase, StudentBrowser

object StudentDatabase extends Database {

  object FrozenFood extends Food("FrozenFood")

  object HeatItUp extends Recipe(
    "heat it up",
  

In [43]:
os.proc("scala", "ch29/GotApples2.scala" , "simple").call()

: 

In [47]:
// val database: db.type = db
println(os.read(os.pwd / "ch29" / "GotApples2.scala"))

/// Food

abstract class Food(val name: String) {
  override def toString = name
}

/// Recipe

class Recipe(
  val name: String,
  val ingredients: List[Food],
  val instructions: String
) {
  override def toString = name
}


/// Database

trait FoodCategories {
  case class FoodCategory(name: String, foods: List[Food])

  def allCategories: List[FoodCategory]
}

abstract class Database extends FoodCategories {
  def allFoods: List[Food]

  def allRecipes: List[Recipe]

  def foodNamed(name: String) =
    allFoods.find(f => f.name == name)
}

/// Browser

abstract class Browser {
  val database: Database

  def recipesUsing(food: Food) =
    database.allRecipes.filter(recipe =>
      recipe.ingredients.contains(food))

  def displayCategory(category: database.FoodCategory) = {
    println(category)
  }
}

/// StudentDatabase, StudentBrowser

object StudentDatabase extends Database {

  object FrozenFood extends Food("FrozenFood")

  object HeatItUp extends Recipe(
    "heat it up",
  

In [49]:
os.proc("scala", "ch29/GotApples2.scala" , "simple").call().out.trim()

res48: String = "fruit salad"

In [50]:
os.proc("scala", "ch29/GotApples2.scala" , "student").call().out.trim()

res49: String = "FoodCategory(edible,List(FrozenFood))"